<div class="alert alert-info">

# Advanced Pandas II — Unstructured Data

Modern data is rarely a clean table. It is multimodal: numbers, text, images, and nested JSON metadata.  
To analyze it efficiently and prepare it for machine learning, we must master data loading, normalization, and feature extraction across different formats.

In this notebook you will work with multiple real-world datasets including retail transactions, company metadata, and real estate listings to explore how pandas and related tools handle structured, semi-structured, textual, and visual data.

</div>

### What you will do

* Load and save tabular data (CSV ↔ Parquet ↔ Excel) while comparing storage size, performance, and dtype preservation.
* Normalize nested JSON into flat tables using `json_normalize`, ready for analysis.
* Preprocess textual descriptions with TF-IDF to transform unstructured text into numerical features.
* Extract image features using Local Binary Patterns (LBP) to represent visual texture numerically.

By the end, you will know how to bring together heterogeneous data sources into one coherent analytical workflow, a key step toward building real, data-driven business insights.

</div>


# Part 1 — Tabular Data and Parquet Efficiency

We start with a dataset that resembles a real business scenario: **Online Retail II**.  
It contains transactions from a UK-based online store, including product descriptions, quantities, prices, customer IDs, and dates.  
This dataset is ideal to study how pandas handles large tabular data and how Parquet improves efficiency in terms of size, speed, and type preservation.

<div class="alert alert-info">

### Exercise 1.0 — Download and Inspect the Data

1. Download the **Online Retail II** dataset from the UCI Machine Learning Repository or Kaggle.  
   You can use the following direct link for convenience:  
   [https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx](https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx)  
2. Save the file in the same folder that this notebook.  
3. Verify that the file is available and readable by pandas.

</div>

In [2]:
from pathlib import Path
Path("online_retail_II.xlsx").exists()

True

<div class="alert alert-info">

### Exercise 1.1 — Load and Explore

1. Read the CSV file into a DataFrame.  
2. Display the first few rows.  
3. Compute the total memory usage with `df.info(memory_usage="deep")`.

</div>

Questions to consider:  
* Which columns contain missing values?  
* Which columns could be treated as categorical or datetime?  
* How much memory does the dataset occupy?

<div class="alert alert-success">

Tip:
The dataset contains two sheets (2010–2011 and 2009–2010).
You can choose one using the sheet_name argument in pd.read_excel, for example:

`pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")`

</div>

In [3]:
import pandas as pd

df = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")
df.head()
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 126.2 MB


In [4]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 126.2 MB


<div class="alert alert-info">

## Exercise 1.2 — Clean and Prepare

1. Convert InvoiceDate to datetime.
2. Drop rows with missing Customer ID.
3. Convert Country to a categorical variable.
4. Recheck memory usage.

</div>

In [ ]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")
df = df.dropna(subset=["Customer ID"])
df["Country"] = df["Country"].astype("category")
df.info(memory_usage="deep")

<div class="alert alert-info">

### Exercise 1.3 — Write, Time, and Check Sizes

1. Save the cleaned DataFrame as  
   * CSV  
   * Parquet with Snappy compression  
   * Parquet with Gzip compression  
2. Measure write times for each format using `%timeit`.  
3. Check on disk file sizes and compare.

</div


Gzip often produces a smaller file than Snappy but takes longer to write.
Snappy usually offers a better balance for analytics workflows.


<div class="alert alert-warning">

Warning:
If you see an error about missing engines for Parquet, install pyarrow in your environment.

</div>

<div class="alert alert-info">
    
### Exercise 1.4 — Read, Time, and Check Memory

1. Measure read times for CSV and both Parquet files using %timeit.
2. After loading each version, call .info(memory_usage="deep") and record total memory use.
3. Confirm that row count and key dtypes match across formats.

</div>

<div class="alert alert-success">

Tip:
Parquet preserves categorical and datetime types more reliably than CSV.
Inspect the `.dtypes` to confirm that `Country` is categorical and `InvoiceDate` is datetime after reading.

</div>

In [ ]:
# READ TIMING

In [ ]:
# MEMORY USAGE

In [ ]:
# INTEGRITY CHECKS

# Part 2 — Companies JSON Lines: Load, Normalize, Clean, and Export to Parquet

You will work with a newline-delimited JSON file of company records.  
Your goals are to load it correctly, normalize nested structures with `pd.json_normalize`, clean and standardize key fields, create a few compact features, validate, and export to Parquet.

<div class="alert alert-success">


**Expected result**  
1. A companies table with one row per company and clean types  
2. Normalized tables for offices, relationships, and funding rounds  
3. An enriched companies table with simple aggregated features

</div>



<div class="alert alert-info">

### Exercise 2.0 — Download the JSON Lines file

1. Open the dataset page  
   https://github.com/ozlerhakan/mongodb-json-files/blob/master/datasets/companies.json  
   Click Raw and save as `companies.json`.  
2. Confirm the file is present before proceeding.

</div>



In [ ]:
from pathlib import Path
Path("companies.json").exists()

<div class="alert alert-info">

### Exercise 2.1 — Load and quick schema audit

1. Load the file using `pd.read_json(..., lines=True)`.  
2. Inspect shape, columns, a few rows, and missingness.  
3. Identify nested candidates for normalization such as `offices`, `relationships`, `funding_rounds`.

</div>

<div class="alert alert-success">

**Tip**  
Newline-delimited JSON files contain one object per line.  Use `lines=True` to read them.

</div>

In [ ]:
# YOUR CODE HERE

<div class="alert alert-info">

### Exercise 2.2 — Build a companies table with clean types

1. Select useful top-level columns such as  
   `permalink`, `name`, `category_code`, `founded_year`, `number_of_employees`, `deadpooled_year`, `total_money_raised`.  
2. Convert years and employees to nullable integers.  
3. Parse `total_money_raised` strings like `"$5M"` into numeric USD amounts.

</div>

In [ ]:
import numpy as np
import re

def parse_money_usd(x):
    if not isinstance(x, str) or not x:
        return np.nan
    s = x.strip().replace(",", "").replace("$", "")
    m = re.fullmatch(r"(?i)(\d*\.?\d+)\s*([km]?)", s)
    if not m:
        return np.nan
    val = float(m.group(1))
    suf = m.group(2).lower()
    if suf == "k":
        val *= 1_000
    elif suf == "m":
        val *= 1_000_000
    return val

In [ ]:
# YOUR CODE HERE

<div class="alert alert-info">

### Exercise 2.3 — Normalize funding rounds with `pd.json_normalize`

1. Expand the `funding_rounds` list so that each round becomes one row.  
2. Keep the parent company keys with `meta=["permalink","name"]`.  
3. Select and standardize key fields such as `round_code`, `funded_year`, `raised_amount`, `raised_currency_code`.  
4. Coerce numeric fields and clean string columns.
5. Select the following columns `["permalink","name","round_code","funded_year","raised_amount","raised_currency_code"]`.

</div>



**Explanation — What `pd.json_normalize` is doing here**

Each company record may contain a **list** under the key `funding_rounds`.  
`pd.json_normalize(..., record_path="funding_rounds", meta=["permalink","name"])` flattens that list so that:

* Each funding round becomes **one row** in the output.  
* `permalink` and `name` are copied from the parent company into every row to preserve the link.  
* The result is a **rectangular table** with scalar columns that pandas can group, aggregate, and join easily.



In [ ]:
# YOUR CODE HERE
funding = pd.json_normalize(
    df,
    record_path="funding_rounds",
    meta=["permalink","name"]
)

funding = funding[[
    "permalink","name","round_code",
    "funded_year","raised_amount","raised_currency_code"
]]

funding["funded_year"] = funding["funded_year"].astype("Int64")
funding["raised_amount"] = funding["raised_amount"].astype("Float64")


<div class="alert alert-info">

### Exercise 2.4 — Aggregate funding features and enrich the companies table

1. Compute per company number of funding rounds, total raised amount for a chosen currency such as USD, and first and last funded years. Do it using a `.groupby` followed by a `.agg`. 
2. Merge these features back into the companies table using a left join on `permalink`.  
3. Cast counts to nullable integers and amounts to float.

</div>

<div class="alert alert-warning">

**Warning**  
If the dataset mixes currencies, aggregating all rounds without filtering can produce inconsistent totals.  
Filter to a single currency or convert amounts before summing.

</div>
<div class="alert alert-danger">

**Pitfall**  
Using an inner join when merging features would drop companies with no funding rounds.  
Use a left join from the companies table to retain companies without funding and keep their feature values as missing.

</div>

In [ ]:
# YOUR CODE HERE
agg = (
    funding[funding["raised_currency_code"] == "USD"]
    .groupby("permalink")
    .agg(
        rounds=("round_code","count"),
        total_usd=("raised_amount","sum"),
        first_year=("funded_year","min"),
        last_year=("funded_year","max")
    )
)

agg["rounds"] = agg["rounds"].astype("Int64")
agg["first_year"] = agg["first_year"].astype("Int64")
agg["last_year"] = agg["last_year"].astype("Int64")

companies_enriched = companies.merge(agg, on="permalink", how="left")
companies_enriched.head()


# Part 3 — Multimodal Data: Text + Images

## Objective

In this section, we will explore how to preprocess **multimodal data** — that is, data combining **textual descriptions** and **property images** — to make it ready for machine learning pipelines.  
We will use a real dataset of property listings that contains both structured attributes and associated photos.

</div>


### Learning goals

By the end of these sections, you should be able to:

1. Load and explore a dataset that includes both text and image data.  
2. Preprocess listing descriptions for text analytics using **TF-IDF**.  
3. Extract simple visual descriptors from images using **Local Binary Patterns (LBP)**.  
4. Combine text, numeric, and image features into a single tabular format ready for modeling.

<div class="alert alert-info">

### Exercise 3.0 — Download the Real Estate dataset

1. Open the dataset page  
   https://huggingface.co/datasets/Binaryy/multimodal-real-estate-search  
   This dataset contains both **property descriptions** and **images**.

2. Make sure the **datasets** library is installed in your environment.

3. Run the cell below to automatically download a small sample and prepare:
- `real_estate/listings.csv` — text and metadata  
- `real_estate/images/` — a few example property images

4. Confirm the files are present before proceeding.
</div>

In [ ]:
from datasets import load_dataset
from pathlib import Path
from PIL import Image
import pandas as pd

root = Path("real_estate")
img_dir = root / "images"
root.mkdir(exist_ok=True)
img_dir.mkdir(exist_ok=True)

# Load and sample 50 listings for fast execution
ds = load_dataset("Binaryy/multimodal-real-estate-search", split="train").select(range(50))

records = []
for i, ex in enumerate(ds):
    img = ex["image"]
    if isinstance(img, Image.Image):
        path = img_dir / f"{i:04d}.jpg"
        img.save(path)
        records.append({
            "listing_id": f"L{i:04d}",
            "city": ex.get("Location", ""),
            "description": (ex.get("Title", "") + " " + ex.get("Details", "")).strip(),
            "image_file": str(path)
        })

df = pd.DataFrame(records)
df.to_csv(root / "listings.csv", index=False)

print(f"Saved {len(df)} listings with images.")
df.head(5)


<div class="alert alert-info">

### Exercise 3.1 — Create TF-IDF features from descriptions

1. Load `real_estate/listings.csv`.  
2. Use `TfidfVectorizer` to transform the text in the `description` column into numerical features.  
3. Keep only the most frequent words and short word pairs (bigrams) to limit size.  
4. Store the resulting matrix as a new DataFrame named `tfidf_df`.

</div>

<div class="alert alert-success">

**Tip**  
Each column in `tfidf_df` now represents the importance of a specific word or bigram in a listing’s description.  
You can later merge a subset of these features (for example, the top ones by variance) with numeric columns like price or city.

</div>


In [ ]:
# YOUR CODE HERE
from sklearn.feature_extraction.text import TfidfVectorizer

text = df["description"].fillna("")

vectorizer = TfidfVectorizer(
    max_features=2000,
    ngram_range=(1,2),
    stop_words="english"
)

X = vectorizer.fit_transform(text)

tfidf_df = pd.DataFrame(
    X.toarray(),
    columns=vectorizer.get_feature_names_out()
)

tfidf_df["listing_id"] = df["listing_id"].values
tfidf_df.head()

,tfidf_24,tfidf_24 hours,tfidf_access,tfidf_accessories,tfidf_accessories modern,tfidf_ajah,tfidf_all,tfidf_all rooms,tfidf_also,tfidf_amenities,...,tfidf_water heater,tfidf_water heaters,tfidf_we,tfidf_well,tfidf_whatsapp,tfidf_with,tfidf_with accessories,tfidf_with bq,tfidf_you,tfidf_your
listing_id,,,,,,,,,,,,,,,,,,,,,
L0000,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.0,0.0,0.000000,0.068978,0.000000,0.000000,0.0000,0.000000
L0001,0.0,0.0,0.0,0.000000,0.000000,0.1427,0.087973,0.100295,0.150067,0.0000,...,0.000000,0.000000,0.0,0.0,0.106277,0.123706,0.000000,0.130690,0.1427,0.136319
L0002,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.092276,0.105200,0.000000,0.0000,...,0.114933,0.000000,0.0,0.0,0.111474,0.000000,0.000000,0.000000,0.0000,0.000000
L0003,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.110230,0.125670,0.000000,0.0000,...,0.000000,0.188034,0.0,0.0,0.133165,0.077502,0.000000,0.163754,0.0000,0.000000
L0004,0.0,0.0,0.0,0.146739,0.152258,0.0000,0.000000,0.000000,0.000000,0.1417,...,0.132773,0.000000,0.0,0.0,0.000000,0.074949,0.152258,0.000000,0.0000,0.000000


<div class="alert alert-info">

### Exercise 4.1 — Build an LBP feature matrix for all images

1. Load `real_estate/listings.csv`.  
2. For each image, convert to grayscale, resize to a fixed shape, and compute **Local Binary Patterns (LBP)**.  
3. Convert the grayscale image to `uint8` before applying LBP to avoid floating-point warnings.  
4. Store the normalized LBP histogram for each image in a feature matrix and save it to Parquet.

</div>

<div class="alert alert-success">

**Tip**  
Converting to `uint8` ensures pixel values are discrete and stable for pattern comparisons.  
With `P=8` and `method="uniform"`, each row of `lbp_df` will have **10 histogram bins** (`P+2`).

</div>

<div class="alert alert-warning">

**Warning**  
Always resize to the same dimensions before feature extraction; differing resolutions change the texture statistics.

</div>

<div class="alert alert-success">

**Tip**  
To resize and convert an image to `uint8` safely:

```python
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.transform import resize
import numpy as np

img = rgb2gray(imread("real_estate/images/0000.jpg"))
img_resized = resize(img, (128, 128), anti_aliasing=True)
img_uint8 = (img_resized * 255).astype("uint8")


In [86]:
# YOUR CODE HERE

,lbp_0,lbp_1,lbp_2,lbp_3,lbp_4,lbp_5,lbp_6,lbp_7,lbp_8,lbp_9
listing_id,,,,,,,,,,
L0000,0.046814,0.076416,0.055725,0.110535,0.209351,0.161133,0.066956,0.078735,0.086792,0.107544
L0001,0.033813,0.067566,0.045410,0.110535,0.214661,0.173828,0.072876,0.088074,0.093811,0.099426
L0002,0.035278,0.084900,0.043396,0.105652,0.226074,0.162109,0.076294,0.072815,0.096130,0.097351
L0003,0.019714,0.085449,0.025208,0.103394,0.246643,0.166382,0.048950,0.102600,0.111450,0.090210
L0004,0.013977,0.043396,0.032288,0.134338,0.288879,0.232361,0.062012,0.066956,0.064026,0.061768



<div class="alert alert-info">

### Exercise 4.2 — Merge text (TF-IDF) + image (LBP) + listings

1. Ensure you have already created:
   - `df` from `real_estate/listings.csv`
   - `tfidf_df` from Exercise 3.1
   - `lbp_df` from Exercise 4.1 (and saved to `real_estate/exports/lbp_features.parquet`)
2. Add `listing_id` to `tfidf_df` and merge everything on `listing_id`.
3. Save the final multimodal table to Parquet.

</div>


In [88]:
# YOUR CODE HERE

Merged shape: (50, 231)


,listing_id,city,description,image_file,lbp_0,lbp_1,lbp_2,lbp_3,lbp_4,lbp_5,...,tfidf_water heater,tfidf_water heaters,tfidf_we,tfidf_well,tfidf_whatsapp,tfidf_with,tfidf_with accessories,tfidf_with bq,tfidf_you,tfidf_your
0,L0000,"Lekki Phase 1, Lekki, Lagos",5 bedroom detached duplex for sale FOR SALE\n\...,real_estate/images/0000.jpg,0.046814,0.076416,0.055725,0.110535,0.209351,0.161133,...,0.000000,0.000000,0.0,0.0,0.000000,0.068978,0.000000,0.000000,0.0000,0.000000
1,L0001,Contemporary 5 Bedroom Detached Duplex With Bq...,5 bedroom detached duplex for sale Newly built...,real_estate/images/0001.jpg,0.033813,0.067566,0.045410,0.110535,0.214661,0.173828,...,0.000000,0.000000,0.0,0.0,0.106277,0.123706,0.000000,0.130690,0.1427,0.136319
2,L0002,"In A Fully Organized Estate, Lekki Phase 1, Le...",4 bedroom block of flats for sale FOR SALE : L...,real_estate/images/0002.jpg,0.035278,0.084900,0.043396,0.105652,0.226074,0.162109,...,0.114933,0.000000,0.0,0.0,0.111474,0.000000,0.000000,0.000000,0.0000,0.000000
3,L0003,"Off Freedom Way, Lekki Phase 1, Lekki, Lagos",6 bedroom detached duplex for sale DETACHED DU...,real_estate/images/0003.jpg,0.019714,0.085449,0.025208,0.103394,0.246643,0.166382,...,0.000000,0.188034,0.0,0.0,0.133165,0.077502,0.000000,0.163754,0.0000,0.000000
4,L0004,"Lekki Phase 1, Lekki, Lagos",3 bedroom terraced duplex for sale BRAND NEW 3...,real_estate/images/0004.jpg,0.013977,0.043396,0.032288,0.134338,0.288879,0.232361,...,0.132773,0.000000,0.0,0.0,0.000000,0.074949,0.152258,0.000000,0.0000,0.000000
